In [7]:
%pip install pymongo
%pip install heapq

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement heapq (from versions: none)
ERROR: No matching distribution found for heapq


In [8]:
import json
from pymongo import MongoClient
from heapq import heapify, heappush, heappop

In [10]:
class ProjectCache:
    
    def __init__(self):
        self.cache = {}                 #Cache
        self.minheap = []               #Minheap
        self.cache_limit_value = 200    #The cache will be set up to only hold 200 tweets
        
    def __contains__(self, key):
        
        return key in self.cache
    
    def current_cache_size(self):
        
        return len(self.cache)
    
    def initial_population(self, database):
        
        #Read from database url and get tweets collection
        client = MongoClient(database)
        db = client["my_final_database"]
        collection_name = db["tweets"]
        #Read every tweet in collection and attempt to add them one by one to cache
        for i in collection_name:
            add_update(i["id"], i)
        
    
    def add_update(self, key, value):
        
        #Add a section that first checks if the current key's relevance value is greater 
        #than the least relevant term in the cache
        if key not in self.cache:
            #If the cache limit is met get the least relevant in cache then compare current item with least relevant in cache
            if len(self.cache) >= self.cache_limit_value:  
                least_relevant_term()                                       
                if value["relevance"] > least_relevant_value_in_cache:      
                    
                    #then remove least relevant to make room in cache
                    self.remove_least_relevant(least_relevant_key, least_relevant_value_in_cache)          
                    
                    #Add key and relevance value to minheap list
                    minheap_sort(key, value["relevance"])
                    #Then add key and all tweet elements to cache
                    self.cache[key] = value                  
        
            else:
                #Add key and relevance value to minheap list
                minheap_sort(key, value["relevance"])
                #Then add key and all tweet elements to cache
                self.cache[key] = value
        
    def remove_least_relevant(self, least_relevant_key, least_relevant_value_in_cache):
        
        #Remove least relevant from both the cache and the heaped list
        self.cache.pop(least_relevant)
        heappop(self.minheap, (least_relevant_value_in_cache, least_relevant))
        heapify(self.minheap)
        
    def least_relevant_term(self):
        
        #Get the least relevant term within the cache as well as key associated with it.      
        least_relevant_value_in_cache = self.minheap[0][0]
        least_relevant_key = self.minheap[0][1]
        return least_relevant_value_in_cache, least_relevant_key
    
    def minheap_sort(self, key, value):
        
        #The relevance will be the new key, and the key will be the value in the minheap.      
        heappush(self.minheap, (value, key))
        heapify(self.minheap)
    